In [66]:
# imports
import os
import tensorflow as tf
import numpy as np
import gradio as gr
import cv2

In [67]:
model = tf.keras.models.load_model("models/Sprint_Start.h5", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_6 (Reshape)             │ (None, 1, 51)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 1, 128)         │        92,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 158,723 (620.02 KB)

 Trainable params: 158,465 (619.00 KB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 2 (12.00 B)

In [68]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def extract_features_from_video(frames):
    # Load a pre-trained CNN
    feature_extractor = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    feature_extractor.trainable = False

    # Preprocess frames
    frames = preprocess_input(frames)  # Normalize frames for MobileNetV2
    features = feature_extractor.predict(frames)  # Extract features

    # Global Average Pooling to reduce dimensionality
    pooled_features = tf.reduce_mean(features, axis=(1, 2))  # Shape: (num_frames, feature_dim)

    # Reduce to 51 dimensions using PCA
    pca = PCA(n_components=51)
    reduced_features = pca.fit_transform(pooled_features.numpy())  # Ensure numpy array
    return reduced_featuresfeatures.numpy()  # Convert to NumPy


In [69]:
from sklearn.decomposition import PCA

# def aggregate_features(features):
#     # Apply PCA to reduce dimensionality to 51 features
#     pca = PCA(n_components=51)
#     aggregated_features = pca.fit_transform(features)  # Shape: (num_frames, 51)
#     return aggregated_features

def aggregate_features(features):
    n_samples, n_features = features.shape
    if n_samples < 51 or n_features < 51:
        # Directly use mean as a fallback for small videos
        mean_features = np.mean(features, axis=0).reshape(1, -1)
        padded_features = np.zeros((1, 51))
        padded_features[0, :mean_features.shape[1]] = mean_features
        return np.repeat(padded_features, n_samples, axis=0)  # Repeat to match samples
    else:
        # Perform PCA with 51 components
        pca = PCA(n_components=51)
        return pca.fit_transform(features)




In [70]:
SPORT_BRANCH_MODEL_MAPPING = {
    "Sprint Start": "models/Sprint_Start.h5",
    "Sprint Running": "models/Sprint.h5",
    "Shot Put": "models/Kogelstonen.h5",
    "Relay Receiver": "models/Estafette.h5",
    "Long Jump": "models/Verspringen.h5",
    "Javelin": "models/Speerwerpen.h5",
    "High Jump": "models/Hoogspringen.h5",
    "Discus Throw": "models/Discurwepen.h5",
    "Hurdling": "models/Hordelopen.h5"
}

In [71]:
def process_video(video_path, sport_branch):
    model_path = SPORT_BRANCH_MODEL_MAPPING.get(sport_branch)
    if not model_path:
        return f"Error: No model found for sport branch: {sport_branch}"
    try:
        model = tf.keras.models.load_model(model_path, custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
    except Exception as e:
        return f"Error loading model: {str(e)}"

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return "Error: Couldn't read video stream from file"

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frame = frame / 255.0
        frames.append(frame)
    cap.release()

    frames = np.array(frames)
    if frames.size == 0:
        return "Error: No frames extracted from video."

    features = extract_features_from_video(frames)

    try:
        aggregated_features = aggregate_features(features)
        print(f"Aggregated features for {video_path}: {aggregated_features[:5]}")  # Log features
    except Exception as e:
        return f"Error during feature aggregation: {str(e)}"

    predictions = model.predict(aggregated_features)
    print(f"Predictions for {video_path}: {predictions[:5]}")  # Log predictions

    final_grade = np.mean(predictions)
    final_grade = np.clip(final_grade, 0, 5)

    return f"Final Grade: {final_grade:.2f} / 5"



In [72]:
def extract_features_from_video(frames):
    # Load a pre-trained CNN
    feature_extractor = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    feature_extractor.trainable = False

    # Preprocess frames
    frames = preprocess_input(frames)  # Normalize frames for MobileNetV2
    features = feature_extractor.predict(frames)  # Extract features

    # Global Average Pooling to reduce dimensionality
    pooled_features = tf.reduce_mean(features, axis=(1, 2))  # Shape: (num_frames, feature_dim)

    # Reduce to 51 dimensions using PCA
    pca = PCA(n_components=51)
    reduced_features = pca.fit_transform(pooled_features.numpy())  # Ensure numpy array
    return reduced_features

In [73]:
def gradio_interface():
    with gr.Blocks() as athletics_app:
        gr.Markdown("# Athletics Evaluation System")

        with gr.Tab("Upload and Process Video"):
            sport_branch_input = gr.Dropdown(
                list(SPORT_BRANCH_MODEL_MAPPING.keys()),
                label="Select Sport Branch *"
            )
            video_input = gr.Video(label="Upload 3D Video * (Ensure proper format)")
            process_btn = gr.Button("Process Video")
            output = gr.Textbox(label="Processing Result", interactive=False)

            def process_and_display(video, sport_branch):
                if not video:
                    return "Error: Please upload a video file."
                if not sport_branch:
                    return "Error: Please select a sport branch."
                return process_video(video, sport_branch)

            process_btn.click(
                process_and_display,
                inputs=[video_input, sport_branch_input],
                outputs=output
            )

    athletics_app.launch(debug=True)


if __name__ == "__main__":
    gradio_interface()


python(61548) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


75/75 ━━━━━━━━━━━━━━━━━━━━ 19s 240ms/step
Aggregated features for /private/var/folders/wy/99dkswn162g8dvl4934_v1rr0000gn/T/gradio/6d018de720eea58f30db5c238742a615b46e8f9fdca9bf5b7895088954027c6a/10000000 6110465729036786 6689576925545987563 n.mp4: [[ 8.53632838e-02 -1.23045444e+00 -2.88924396e-01  8.26340020e-01
  -2.08476231e-01  2.04020739e-01  1.52514884e-02  2.37199385e-02
  -3.62466526e-04  1.19489490e-03  3.67494076e-02 -3.55127901e-02
  -5.14904149e-02  2.10678801e-02 -5.03515126e-03 -6.72045397e-03
  -6.15637610e-03 -6.31009927e-03 -1.82876771e-03 -2.41852482e-03
   2.11959332e-03 -2.56938906e-03 -2.64676474e-03  6.12409378e-04
  -4.64173360e-03  2.77493009e-03 -1.54450070e-03  5.23407361e-04
   2.55867373e-03 -4.86786943e-04 -3.44943575e-04  3.18887207e-04
  -1.58915389e-03  5.00214286e-04 -5.66194183e-04 -1.51130790e-03
  -7.85136945e-04  5.73512698e-05 -7.46871083e-05 -6.30332332e-04
  -2.75992032e-04 -9.44480242e-04  2.88610405e-04 -1.73483801e-04
   8.14009854e-06  4.05672